In [18]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, TensorDataset
import copy
import sys, os
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import matplotlib.pyplot as plt
import math
import sklearn
from sklearn.metrics import r2_score
from scipy.io import savemat
np.set_printoptions(linewidth=500)

In [19]:
import sys
print(sys.version)

3.6.10 (default, Dec 19 2019, 23:04:32) 
[GCC 5.4.0 20160609]


# Noise가 X에 dependent 하다고 가정하고 CGAN 학습하는 코드
# P-type 만 가지고 .. 

In [20]:
seed = 0

# excel file 보고 값 기입
n_of_cycle = 36 # 조합 갯수 : n_of_cycle
n_in_cycle = 50 # random variation 갯수

In [21]:
torch.manual_seed(seed)
# torch.cuda.manual_seed(seed)
np.random.seed(seed)
#random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [28]:
def load_data(path, sheet_name, num_of_cycle, num_in_cycle):

    X_dim = 3
    x_df = pd.read_excel(path, sheet_name=sheet_name, header=0, usecols="D:F")
    
    X_total = np.zeros((num_of_cycle, X_dim)) 
    for cycle in range(num_of_cycle):
        X_total[cycle] = x_df[cycle*num_in_cycle+1:cycle*num_in_cycle+2].values
    X_total_expanded = np.repeat(X_total,num_in_cycle,axis=0)
    
    
    #-------------------------------------------------------------------------------
    Y_dim = 9
    y_df = pd.read_excel(path, sheet_name=sheet_name, header=0, usecols="H:P")

    Y_total = np.zeros((num_of_cycle*num_in_cycle, Y_dim)) 
    for i in range(num_of_cycle*num_in_cycle):
        Y_total[i] = y_df[i+1:i+2].values
    
    return (X_total, X_total_expanded, Y_total)

In [27]:
# 파일로부터 데이터불러오기
X_total, X_total_expanded, Y_total_= load_data(path='./LER_20200529_V004_edit.xlsx', sheet_name='Generated DATAs', num_of_cycle=72, num_in_cycle=n_in_cycle)
print(X_total.shape, X_total_expanded.shape, Y_total_.shape)

odd_index_for_ptype = np.array([2*i+1 for i in range(n_of_cycle)])
X_total = X_total[odd_index_for_ptype]
X_total_expanded = np.repeat(X_total,n_in_cycle,axis=0)

Y_total = np.zeros((n_of_cycle*n_in_cycle, Y_total_.shape[1]))
for i in range(len(odd_index_for_ptype)):
    Y_total[i*n_in_cycle:(i+1)*n_in_cycle]=Y_total_[odd_index_for_ptype[i]*n_in_cycle:(odd_index_for_ptype[i]+1)*n_in_cycle]



X_data = copy.deepcopy(X_total)
print(X_data.shape)

y_data = copy.deepcopy(Y_total)
print(y_data.shape)
print('y값 nan 있는 곳 있나? : ', np.argwhere(np.isnan(y_data)))

ValueError: could not broadcast input array from shape (4) into shape (3)

In [13]:
# 파일로부터 데이터불러오기
X_total, X_total_expanded, Y_total_= load_data(path='./2020_LER_20200529_V004.xlsx', sheet_name='uniformly sampling', num_of_cycle=72, num_in_cycle=n_in_cycle)
print(X_total.shape, X_total_expanded.shape, Y_total_.shape)

odd_index_for_ptype = np.array([2*i+1 for i in range(n_of_cycle)])
X_total = X_total[odd_index_for_ptype]
X_total_expanded = np.repeat(X_total,n_in_cycle,axis=0)

Y_total = np.zeros((n_of_cycle*n_in_cycle, Y_total_.shape[1]))
for i in range(len(odd_index_for_ptype)):
    Y_total[i*n_in_cycle:(i+1)*n_in_cycle]=Y_total_[odd_index_for_ptype[i]*n_in_cycle:(odd_index_for_ptype[i]+1)*n_in_cycle]



X_data = copy.deepcopy(X_total)
print(X_data.shape)

y_data = copy.deepcopy(Y_total)
print(y_data.shape)
print('y값 nan 있는 곳 있나? : ', np.argwhere(np.isnan(y_data)))

(72, 4) (3600, 4) (3600, 9)
(36, 4)
(1800, 9)
y값 nan 있는 곳 있나? :  []


In [7]:
X_total.shape

(36, 3)

In [15]:
# 1. mean 예측하는 모델만들어야함.
y_mean = np.mean(np.split(y_data, n_of_cycle), axis = 1)
#y_std = np.std(np.split(y_data, n_of_cycle), axis = 1)
print(y_mean.shape)

# just check
print(np.min(y_mean,axis=0))
print(np.max(y_mean,axis=0))

(36, 9)
[ 2.63792000e-12  2.34130000e-05  6.97438000e-06  5.77404800e-07  2.20184000e-05 -3.50360000e-01 -3.63020000e-01 -6.24891600e+01 -1.97230769e+01]
[ 3.93564000e-12  2.63074000e-05  7.70604000e-06  7.46248000e-07  2.47592000e-05 -3.39000000e-01 -3.51820000e-01 -6.20826800e+01 -1.75076923e+01]


In [16]:
# Y-Y_mean을 통해 noise만 만들어낼꺼임.. shape 맞추기
y_mean_expanded = np.repeat(y_mean, n_in_cycle, axis=0)
noise_total = y_data - y_mean_expanded
print(y_mean_expanded)

[[ 2.92828000e-12  2.59490000e-05  7.49488000e-06 ... -3.53800000e-01 -6.21401400e+01 -1.81230769e+01]
 [ 2.92828000e-12  2.59490000e-05  7.49488000e-06 ... -3.53800000e-01 -6.21401400e+01 -1.81230769e+01]
 [ 2.92828000e-12  2.59490000e-05  7.49488000e-06 ... -3.53800000e-01 -6.21401400e+01 -1.81230769e+01]
 ...
 [ 2.88569400e-12  2.45318000e-05  7.24998000e-06 ... -3.58820000e-01 -6.21613600e+01 -1.88000000e+01]
 [ 2.88569400e-12  2.45318000e-05  7.24998000e-06 ... -3.58820000e-01 -6.21613600e+01 -1.88000000e+01]
 [ 2.88569400e-12  2.45318000e-05  7.24998000e-06 ... -3.58820000e-01 -6.21613600e+01 -1.88000000e+01]]


## Data Split

In [10]:
# cycle을 split하자
train_num = 25
val_num = 5
test_num = 6

In [11]:
#데이터 스플릿
X_train = X_data[:train_num]
X_val = X_data[train_num:train_num+val_num]
X_test = X_data[train_num+val_num: train_num + val_num + test_num]

y_train = y_data[:train_num*n_in_cycle]
y_val = y_data[train_num*n_in_cycle:(train_num+val_num)*n_in_cycle]
y_test = y_data[(train_num+val_num)*n_in_cycle : (train_num + val_num + test_num)*n_in_cycle]

y_mean_train = y_mean[:train_num]
y_mean_val = y_mean[train_num:train_num+val_num]
y_mean_test = y_mean[train_num+val_num: train_num + val_num + test_num]

print(len(X_train), len(X_val), len(X_test))
print(len(y_mean_train), len(y_mean_val), len(y_mean_test))

noise_train = noise_total[:train_num*n_in_cycle]
noise_val = noise_total[train_num*n_in_cycle:(train_num+val_num)*n_in_cycle]
noise_test = noise_total[(train_num+val_num)*n_in_cycle: (train_num + val_num + test_num)*n_in_cycle]

25 5 6
25 5 6


# Data Visualization

In [12]:
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

ModuleNotFoundError: No module named 'seaborn'

## Visualize Y

In [ ]:
y_train_pd = pd.DataFrame({'Column0': y_train[:, 0], 'Column1': y_train[:, 1], 'Column2': y_train[:, 2],
                             'Column3': y_train[:, 3], 'Column4': y_train[:, 4], 'Column5': y_train[:, 5],
                            'Column6': y_train[:, 6], 'Column7': y_train[:, 7], 'Column8': y_train[:, 8]
                              })

In [ ]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(y_train_pd.values)
y_train_pd['pca-one'] = pca_result[:,0]
y_train_pd['pca-two'] = pca_result[:,1] 
y_train_pd['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
cycle_for_coloring = np.zeros(len(y_train_pd), dtype=int)
for i in range(train_num):
    for j in range(n_in_cycle):
        cycle_for_coloring[n_in_cycle*i + j] = i
y_train_pd['cycle'] = cycle_for_coloring

In [ ]:
for i in range(5):
    current_cycle = i
    # 2-dim visualization of noise
    flatui = ["red"]#, "#9b59b6"]#, "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
    plt.figure(figsize=(6,4))
    sns.scatterplot(
        x="pca-one", y="pca-two",
        hue="cycle", 
        palette=sns.color_palette(flatui),
        data=y_train_pd.loc[current_cycle*50:current_cycle*50+50-1,:],
        legend="full",
        alpha=1.0
    )
    plt.xlim(-20,20)
    plt.ylim(-5,5)
    plt.title('y_train for cycle : '+str(i), fontsize=15)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

## Visualize Y_mean

In [ ]:
y_mean_train_pd = pd.DataFrame({'Column0': y_mean_train[:, 0], 'Column1': y_mean_train[:, 1], 'Column2': y_mean_train[:, 2],
                             'Column3': y_mean_train[:, 3], 'Column4': y_mean_train[:, 4], 'Column5': y_mean_train[:, 5],
                            'Column6': y_mean_train[:, 6], 'Column7': y_mean_train[:, 7], 'Column8': y_mean_train[:, 8]
                              })

In [ ]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(y_mean_train_pd.values)
y_mean_train_pd['pca-one'] = pca_result[:,0]
y_mean_train_pd['pca-two'] = pca_result[:,1] 
y_mean_train_pd['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
cycle_for_coloring = np.zeros(len(y_mean_train_pd), dtype=int)
for i in range(train_num):
    cycle_for_coloring[i] = i
y_mean_train_pd['cycle'] = cycle_for_coloring

In [ ]:
# 2-dim visualization of noise
number_of_colors = np.int_(np.max(y_mean_train_pd['cycle'].values))+1
plt.figure(figsize=(12,15))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="cycle",
    palette=sns.color_palette("Paired", number_of_colors),
    data=y_mean_train_pd.loc[:,:],
    legend="full",
    alpha=1.0
)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('y_mean_train', fontsize=20)

## Visualize Noise (PCA+TSNE to 2dim / PCA+TSNE to 3dim / UMAP)

In [ ]:
noise_train_pd = pd.DataFrame({'Column0': noise_train[:, 0], 'Column1': noise_train[:, 1], 'Column2': noise_train[:, 2],
                             'Column3': noise_train[:, 3], 'Column4': noise_train[:, 4], 'Column5': noise_train[:, 5],
                            'Column6': noise_train[:, 6], 'Column7': noise_train[:, 7], 'Column8': noise_train[:, 8]
                              })

In [ ]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(noise_train_pd.values)
noise_train_pd['pca-one'] = pca_result[:,0]
noise_train_pd['pca-two'] = pca_result[:,1] 
noise_train_pd['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
cycle_for_coloring = np.zeros(len(noise_train_pd), dtype=int)
for i in range(train_num):
    for j in range(n_in_cycle):
        cycle_for_coloring[n_in_cycle*i + j] = i
noise_train_pd['cycle'] = cycle_for_coloring

In [ ]:
noise_train_pd.head(5)

In [ ]:
# 2-dim visualization of noise
number_of_colors = np.int_(np.max(noise_train_pd['cycle'].values))+1
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="pca-one", y="pca-two",
    hue="cycle",
    palette=sns.color_palette("Paired", number_of_colors),
    data=noise_train_pd.loc[:,:],
    legend="full",
    alpha=1.0
)
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.title('noise_train', fontsize=20)

In [ ]:
for i in range(5):
    current_cycle = i
    # 2-dim visualization of noise
    flatui = ["red"]#, "#9b59b6"]#, "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
    plt.figure(figsize=(6,4))
    sns.scatterplot(
        x="pca-one", y="pca-two",
        hue="cycle",
        palette=sns.color_palette(flatui),
        data=noise_train_pd.loc[current_cycle*25:current_cycle*25+25-1,:],
        legend="full",
        alpha=1.0
    )
    plt.xlim(-6,6)
    plt.ylim(-1,1)
    plt.title('noise for cycle : '+str(i), fontsize=15)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# noise가 X에 dependent 하는 것 같기도 하다 ?

In [ ]:
# 3-dimensional visualization
ax = plt.figure(figsize=(16,10)).gca(projection='3d')
ax.scatter(
    xs=noise_train_pd.loc[:,:]["pca-one"], 
    ys=noise_train_pd.loc[:,:]["pca-two"], 
    zs=noise_train_pd.loc[:,:]["pca-three"], 
    c=noise_train_pd.loc[:,:]["cycle"], 
    cmap='tab20'
)
ax.set_xlabel('pca-one')
ax.set_ylabel('pca-two')
ax.set_zlabel('pca-three')
plt.show()

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler

In [ ]:
reducer = umap.UMAP()
scaled_noise_train_pd = StandardScaler().fit_transform(noise_train_pd)
embedding = reducer.fit_transform(scaled_noise_train_pd)
embedding.shape

In [ ]:
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=[sns.color_palette("Set1", n_colors=50, desat=.9)[x] for x in noise_train_pd.cycle.values])
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of noise train data', fontsize=10)

# Data Normalization for Training

In [ ]:
# def normalizing(data, params):
#     tr_min, tr_max = params
#     normalized = (data-tr_min)/(tr_max-tr_min)
#     return normalized

In [17]:
# 정규화 계수 구하기
#params = (np.min(y_mean_var_train, axis=0), np.max(y_mean_var_train, axis=0))
tr_x_mean = np.mean(X_train, axis=0, dtype=np.float32) 
tr_x_std = np.std(X_train, axis=0, dtype=np.float32)
tr_y_mean = np.mean(y_mean_train, axis=0, dtype=np.float32) 
tr_y_std = np.std(y_mean_train, axis=0, dtype=np.float32)
print(tr_x_mean, tr_x_std)
print(tr_y_mean, tr_y_std)
#tr_y_mean[0:6] = np.mean(y_data[:2300], axis=0, dtype=np.float32) 
#tr_y_std[0:6] = np.std(y_data[:2300], axis=0, dtype=np.float32)
#print(tr_x_mean, tr_x_std)
#print(tr_y_mean, tr_y_std)

# 데이터 정규화
X_train_normalized = (X_train - tr_x_mean) / tr_x_std
X_val_normalized = (X_val - tr_x_mean) / tr_x_std
X_test_normalized = (X_test - tr_x_mean) / tr_x_std
y_mean_train_normalized = (y_mean_train - tr_y_mean) / tr_y_std
y_mean_val_normalized = (y_mean_val - tr_y_mean) / tr_y_std
y_mean_test_normalized = (y_mean_test - tr_y_mean) / tr_y_std

# y_mean_var_train_norm = normalizing(y_mean_var_train, params)
# y_mean_var_val_norm = normalizing(y_mean_var_val, params)

NameError: name 'X_train' is not defined

# Training NN predicting y_mean

In [ ]:
class Net(nn.Module):
    def __init__(self, hidden_dim):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, 9)
        
    def forward(self, x):
        input_x = x
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=F.relu(self.fc3(x))
        #x=F.dropout(x, training=self.training)
        x=self.fc4(x)
        return x
    
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.kaiming_normal_(m.weight)
            

In [ ]:
def train(model, train_loader, val_loader, optimizer, scheduler, epochs=100, verbose=True):
    criterion = torch.nn.MSELoss()
    #optimizer = optim.SGD(model.parameters(), lr = 0.00001)
    train_loss_list = []
    val_loss_list = []
    r2_score_list = []
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        train_num=0
        for i, data in enumerate(train_loader):
            train_x, train_y = data
            train_x, train_y = train_x.to(device), train_y.to(device)
            
            optimizer.zero_grad()
            output = model(train_x)
            loss = criterion(output, train_y) #순서맞음
            loss.backward()
            optimizer.step()
            train_loss += loss
            
            train_num += len(train_y)
            
        
        
        model.eval()
        val_loss = 0.0
        val_num = 0
        
        val_target = torch.tensor([]).float().to(device)
        val_pred = torch.tensor([]).float().to(device)
        
        for i, data in enumerate(val_loader):
            val_out=[]
            with torch.no_grad():
                val_x, val_y = data
                val_x, val_y = val_x.to(device), val_y.to(device)
                
                val_output = model(val_x)
                val_loss += criterion(val_output, val_y)

                val_target = torch.cat((val_target, val_y))
                val_pred = torch.cat((val_pred, val_output))
                
                val_num += len(val_y)

        
        
        train_loss /= train_num
        val_loss /= val_num   
        r2 = r2_score(val_target, val_pred) # 순서맞음
        
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        r2_score_list.append(r2)
        
        for param_group in optimizer.param_groups:  #scheduler
            lr = param_group['lr']
            #print('lr_schedule:',lr)
        scheduler.step()
        
        if verbose==True :
            if((epoch+1)% 10 == 0):
                print("epoch:{:2d}, lr:{:.6f}, || train_loss:{:.6f}, val_loss:{:.6f}, r2_score:{:.6f}"
              .format(epoch, lr, train_loss, val_loss, r2))
    
    if verbose==False :        
        print("final results : epoch:{:2d}, lr:{:.6f}, || train_loss:{:.6f}, val_loss:{:.6f}, r2_score:{:.6f}"
              .format(epoch, lr, train_loss, val_loss, r2))        
    return model, train_loss_list, val_loss_list, r2_score_list
    
            

In [ ]:
# tensor로..
X_train_normalized = torch.tensor(X_train_normalized).float()
X_val_normalized = torch.tensor(X_val_normalized).float()
y_mean_train_normalized = torch.tensor(y_mean_train_normalized).float()
y_mean_val_normalized = torch.tensor(y_mean_val_normalized).float()

train_dataset = TensorDataset(X_train_normalized, y_mean_train_normalized)
val_dataset = TensorDataset(X_val_normalized, y_mean_val_normalized)

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 32, shuffle = False)

In [ ]:
# model = Net(hidden_dim=100)
# model.apply(init_normal)
# #device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device='cpu'
# model.to(device)
# optimizer = optim.Adam(model.parameters(), lr = 5e-5)
# #optimizer = optim.SGD(model.parameters(), lr = 5e-4, momentum=0.9)
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5) # scheduler
# model, train_loss_list, val_loss_list, r2_scores = train(model, train_loader, val_loader,optimizer, exp_lr_scheduler, 
#                                                          epochs=1000, verbose=True)

In [ ]:
model = Net(hidden_dim=100)
model.apply(init_normal)
#device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device='cpu'
model.to(device)
optimizer = optim.Adam(model.parameters(), lr = 5e-5)
#optimizer = optim.SGD(model.parameters(), lr = 5e-4, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5) # scheduler
model, train_loss_list, val_loss_list, r2_scores = train(model, train_loader, val_loader,optimizer, exp_lr_scheduler, 
                                                         epochs=1000, verbose=True)

In [ ]:
plt.title("train loss")
plt.plot(np.array(train_loss_list), 'r')

In [ ]:
plt.title("val loss")
plt.plot(np.array(val_loss_list), 'r')

In [ ]:
plt.title("r2_score")
plt.ylim(-2,1)
plt.plot(np.array(r2_scores),'r')

# Predict Y_mean (with inverting normalization constant)

In [ ]:
# Empirical mean 보다는 좀 낫다.

In [ ]:
X_test_normalized = torch.tensor(X_test_normalized).float()

In [ ]:
X_test_normalized

In [ ]:
model.eval()
pred = model(X_test_normalized.to(device))
output = np.array(pred.tolist())


In [ ]:
output.shape

In [ ]:
#output inverse 연산 (최종 예측한 Y_mean)
print(tr_x_mean)
print(tr_x_std)
print(tr_y_mean)
print(tr_y_std)

mean_output = output*tr_y_std + tr_y_mean            #(tr_max - tr_min) + tr_min

# Visualize prediction vs target for Y_mean

In [ ]:
y_mean_test_pd = pd.DataFrame({'Column0': y_mean_test[:, 0], 'Column1': y_mean_test[:, 1], 'Column2': y_mean_test[:, 2],
                             'Column3': y_mean_test[:, 3], 'Column4': y_mean_test[:, 4], 'Column5': y_mean_test[:, 5],
                            'Column6': y_mean_test[:, 6], 'Column7': y_mean_test[:, 7], 'Column8': y_mean_test[:, 8]
                              })
y_mean_test_predicted_pd = pd.DataFrame({'Column0': mean_output[:, 0], 'Column1': mean_output[:, 1], 'Column2': mean_output[:, 2],
                             'Column3': mean_output[:, 3], 'Column4': mean_output[:, 4], 'Column5': mean_output[:, 5],
                            'Column6': mean_output[:, 6], 'Column7': mean_output[:, 7], 'Column8': mean_output[:, 8]
                              })

In [ ]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(y_mean_test_pd.values)
y_mean_test_pd['pca-one'] = pca_result[:,0]
y_mean_test_pd['pca-two'] = pca_result[:,1] 
y_mean_test_pd['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

pca = PCA(n_components=3)
pca_result = pca.fit_transform(y_mean_test_predicted_pd.values)
y_mean_test_predicted_pd['pca-one'] = pca_result[:,0]
y_mean_test_predicted_pd['pca-two'] = pca_result[:,1] 
y_mean_test_predicted_pd['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
cycle_for_coloring = np.zeros(len(y_mean_test_pd), dtype=int)
for i in range(test_num):
    cycle_for_coloring[i] = i
y_mean_test_pd['cycle'] = cycle_for_coloring
y_mean_test_predicted_pd['cycle'] = cycle_for_coloring

In [ ]:
for i in range(5):
    current_cycle = i
    # 2-dim visualization of noise
    
    plt.figure(figsize=(6,4))
    sns.scatterplot(
        x="pca-one", y="pca-two",
        hue="cycle",
        palette=sns.color_palette(["red"]),
        data=y_mean_test_pd.loc[current_cycle:current_cycle,:],
        legend="full",
        alpha=1.0
    )
    sns.scatterplot(
        x="pca-one", y="pca-two",
        hue="cycle",
        palette=sns.color_palette(["blue"]),
        data=y_mean_test_predicted_pd.loc[current_cycle:current_cycle,:],
        legend="full",
        alpha=1.0
    )
    
    plt.xlim(-2,2)
    plt.ylim(-0.5,0.5)
    plt.title('y_mean for cycle : '+str(i), fontsize=15)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
y_mean_test_normalized

In [ ]:
output

# Generate Noise with GAN

In [ ]:
def sample_z(batch_size = 1, d_noise=100):    # latent dim 크기를 가지는 가우시안 노이즈 벡터를 배치수만큼의 갯수로 추출하는 함수.
    return torch.randn(batch_size, d_noise, device=device)

In [ ]:
class G(nn.Module):
    def __init__(self, d_noise, d_hidden, dim_output):
        super(G, self).__init__()
        self.fc1 = nn.Linear(d_noise, d_hidden)
        self.fc2 = nn.Linear(d_hidden, d_hidden)
        self.fc3 = nn.Linear(d_hidden, dim_output)
        

    def forward(self, x):
        input_x = x
        x=F.relu(self.fc1(x))
        #x=F.dropout(x, p=0.1)
        x=F.relu(self.fc2(x))
        #x=F.dropout(x, p=0.1)
        x=self.fc3(x)
        output = x
        return x
    
    
class D(nn.Module):
    def __init__(self, dim_output, d_hidden):
        super(D, self).__init__()
        self.fc1 = nn.Linear(dim_output, d_hidden)
        self.fc2 = nn.Linear(d_hidden, d_hidden)
        self.fc3 = nn.Linear(d_hidden, 1)
        

    def forward(self, x):
        input_x = x
        x=F.relu(self.fc1(x))
        #x=F.dropout(x, p=0.1)
        x=F.relu(self.fc2(x))
        #x=F.dropout(x, p=0.1)
        x=torch.sigmoid(self.fc3(x))
        output = x
        return x

In [ ]:
dim_input = 10
dim_output = 9
d_hidden = 10

#CONDITIONAL
G = G(dim_input+3, d_hidden, dim_output)
D = D(dim_output+3, d_hidden)
            

G.to(device)
D.to(device)

In [7]:
# GANs 학습에 사용되는 함수


def run_epoch(generator, discriminator, _optimizer_g, _optimizer_d, train_data_loader):
    
    generator.train()
    discriminator.train()

    for real_batch, X_batch in train_data_loader:
        real_batch, X_batch = real_batch.to(device), X_batch.to(device)
        
        batch_size=len(real_batch)
        
        input_for_D = torch.cat((real_batch,X_batch),axis=1)
        input_for_G = torch.cat((sample_z(batch_size, dim_input),X_batch),axis=1)
        
        p_real = discriminator(input_for_D)
        p_fake = discriminator(torch.cat((generator(input_for_G), X_batch),axis=1))


        loss_real = -1 * torch.log(p_real)   # -1 for gradient ascending
        loss_fake = -1 * torch.log(1.-p_fake) # -1 for gradient ascending
        loss_d    = (loss_real + loss_fake).mean() # 배치에 대해 mean
        
        
        # Update parameters
        _optimizer_d.zero_grad() #grad 계산 전 항상 init 해주기
        loss_d.backward() # grad 계산
        _optimizer_d.step() # update

        
        input_for_G = torch.cat((sample_z(batch_size, dim_input),X_batch),axis=1)
        p_fake = discriminator(torch.cat((generator(input_for_G), X_batch),axis=1))
                
        # instead of: torch.log(1.-p_fake).mean() <- explained in Section 3
        loss_g = -1 * torch.log(p_fake).mean()  # log(D(G(z)))를 maximize 하는 것으로 (초기 D의 overpower 문제 해결위해)
        
        # Update parameters
        _optimizer_g.zero_grad() #grad 계산 전 항상 init 해주기
        loss_g.backward() #grad 계산
        _optimizer_g.step()
          
def evaluate_model(generator, discriminator, test_data_loader):
    
    p_real, p_fake = 0.,0.
    
    generator.eval() # 모드 변경
    discriminator.eval() # 모드 변경
    
    batch_num=0
    for real_batch, X_batch in test_data_loader:
        
        real_batch, X_batch = real_batch.to(device), X_batch.to(device)
        batch_size = len(real_batch)
        
        with torch.autograd.no_grad():
            input_for_D = torch.cat((real_batch,X_batch),axis=1)
            input_for_G = torch.cat((sample_z(batch_size, dim_input),X_batch),axis=1)
            p_real += torch.sum(discriminator(input_for_D)).item()/len(real_batch)
            p_fake += torch.sum(discriminator(torch.cat((generator(input_for_G), X_batch),axis=1))).item()/len(real_batch)
            
        batch_num += 1
    p_real /= batch_num
    p_fake /= batch_num
    return p_real, p_fake

In [8]:
tr_noise_mean = np.mean(noise_train, axis=0, dtype=np.float32) 
tr_noise_std = np.std(noise_train, axis=0, dtype=np.floa t32)

SyntaxError: invalid syntax (<ipython-input-8-3352ae442eae>, line 2)

In [ ]:
noise_train_normalized = (noise_train-tr_noise_mean)/tr_noise_std
noise_val_normalized = (noise_val-tr_noise_mean)/tr_noise_std
noise_test_normalized = (noise_test-tr_noise_mean)/tr_noise_std

In [ ]:
train_dataset_GAN = TensorDataset(torch.tensor(noise_train_normalized).float(),torch.tensor(np.repeat(X_train_normalized.numpy(), 50, axis=0)).float())
val_dataset_GAN = TensorDataset(torch.tensor(noise_val_normalized).float(),torch.tensor(np.repeat(X_val_normalized.numpy(), 50, axis=0)).float())

train_loader_GAN = DataLoader(train_dataset_GAN, batch_size = 64, shuffle = True)
val_loader_GAN = DataLoader(val_dataset_GAN, batch_size = 64, shuffle = False)

In [ ]:
optimizer_g = torch.optim.Adam(G.parameters(), lr = 0.0001)
optimizer_d = torch.optim.Adam(D.parameters(), lr = 0.0005)

p_real_trace = []
p_fake_trace = []

In [ ]:
def init_params(model): # weight initialization 인듯
    for p in model.parameters():
        if(p.dim() > 1):
            nn.init.xavier_normal_(p)
        else:
            nn.init.uniform_(p, 0.1, 0.2)   

init_params(G)
init_params(D)

In [ ]:
scheduler_GAN = lr_scheduler.StepLR(optimizer_d, step_size=50, gamma=0.5) # scheduler

for epoch in range(200):
    
    #print('epoch', epoch)
    
    run_epoch(G, D, optimizer_g, optimizer_d, train_loader_GAN)
    p_real, p_fake = evaluate_model(G, D, val_loader_GAN)
    
    p_real_trace.append(p_real)
    p_fake_trace.append(p_fake) 
    
    
    for param_group in optimizer_d.param_groups:  #scheduler
        lr_d = param_group['lr']
        #print('lr_schedule:',lr)
    scheduler_GAN.step()
    
    
    if((epoch+1)% 10 == 0):
        print('epoch : (%i/200), lr_d:%.10f, p_real: %f, p_g: %f' % (epoch+1, lr_d, p_real, p_fake))

In [ ]:
# epoch이 진행됨에 따라 D(x)와 D(G(z))의 확률값은 어떻게 개선되어 가는지?
plt.plot(p_fake_trace, label='D(x_generated)')
plt.plot(p_real_trace, label='D(x_real)')
plt.legend(bbox_to_anchor=(0.60, 0.95), loc=2, borderaxespad=0.)

#plt.savefig(name+'_G&D_probability.png', dpi=300)
plt.show()


# Visualize prediction vs target for Noise

In [ ]:
input_for_G = torch.cat((sample_z(len(noise_test), dim_input),torch.from_numpy(np.repeat(X_test_normalized.numpy(), 50, axis=0))),axis=1)
noise_test_normalized_predicted = G(input_for_G).detach().cpu().numpy()
noise_test_predicted = noise_test_normalized_predicted*tr_noise_std+tr_noise_mean

In [ ]:
noise_test_pd = pd.DataFrame({'Column0': noise_test[:, 0], 'Column1': noise_test[:, 1], 'Column2': noise_test[:, 2],
                             'Column3': noise_test[:, 3], 'Column4': noise_test[:, 4], 'Column5': noise_test[:, 5],
                            'Column6': noise_test[:, 6], 'Column7': noise_test[:, 7], 'Column8': noise_test[:, 8]
                              })

noise_test_predicted_pd = pd.DataFrame({'Column0': noise_test_predicted[:, 0], 'Column1': noise_test_predicted[:, 1], 'Column2': noise_test_predicted[:, 2],
                             'Column3': noise_test_predicted[:, 3], 'Column4': noise_test_predicted[:, 4], 'Column5': noise_test_predicted[:, 5],
                            'Column6': noise_test_predicted[:, 6], 'Column7': noise_test_predicted[:, 7], 'Column8': noise_test_predicted[:, 8]
                              })

In [ ]:
pca = PCA(n_components=3)
pca_result = pca.fit_transform(noise_test_pd.values)
noise_test_pd['pca-one'] = pca_result[:,0]
noise_test_pd['pca-two'] = pca_result[:,1] 
noise_test_pd['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

pca = PCA(n_components=3)
pca_result = pca.fit_transform(noise_test_predicted_pd.values)
noise_test_predicted_pd['pca-one'] = pca_result[:,0]
noise_test_predicted_pd['pca-two'] = pca_result[:,1] 
noise_test_predicted_pd['pca-three'] = pca_result[:,2]
print('Explained variation per principal component: {}'.format(pca.explained_variance_ratio_))

In [ ]:
cycle_for_coloring = np.zeros(len(noise_test_pd), dtype=int)
for i in range(test_num):
    for j in range(n_in_cycle):
        cycle_for_coloring[n_in_cycle*i + j] = i
noise_test_pd['cycle'] = cycle_for_coloring
noise_test_predicted_pd['cycle'] = cycle_for_coloring

In [ ]:
for i in range(5):
    current_cycle = i
    # 2-dim visualization of noise
    flatui = ["red"]#, "#9b59b6"]#, "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
    plt.figure(figsize=(6,4))
    sns.scatterplot(
        x="pca-one", y="pca-two",
        hue="cycle",
        palette=sns.color_palette(['red']),
        data=noise_test_pd.loc[current_cycle*50:current_cycle*50+50-1,:],
        legend="full",
        alpha=1.0
    )
    sns.scatterplot(
        x="pca-one", y="pca-two",
        hue="cycle",
        palette=sns.color_palette(['blue']),
        data=noise_test_predicted_pd.loc[current_cycle*50:current_cycle*50+50-1,:],
        legend="full",
        alpha=1.0
    )
    plt.xlim(-6,6)
    plt.ylim(-1,1)
    plt.title('noise for cycle : '+str(i), fontsize=15)
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Sample Generation

In [ ]:
y_test_generated = np.repeat(mean_output, 50, axis=0)+ noise_test_predicted

In [ ]:
y_test_generated[0:60]

In [ ]:
y_test[0:60]

In [ ]:
saving_list = {'y_test_generated':y_test_generated, 'y_test':y_test}
saving_list = dict(saving_list)
savemat('./dependent_noise_results.mat', saving_list)

In [ ]:
np.savetxt('dependent_noise_generated_sample.txt', y_test_generated, delimiter=',')

In [ ]:
def mse(predictions, targets):
    return ((predictions - targets) ** 2).mean()
print(mse(y_test_generated, y_test))

In [ ]:
y_test.shape

In [ ]:
plt.scatter(y_test[:50,0],y_test[:50,5])
plt.scatter(y_test_generated[:50,0],y_test[:50,5])
plt.xlim(1e-12,13e-12)
plt.show()

In [ ]:
plt.scatter(y_test[50:100,0],y_test[50:100,5])
plt.scatter(y_test_generated[50:100,0],y_test[50:100,5])
plt.xlim(1e-12,13e-12)
plt.show()

In [ ]:
plt.scatter(y_test[200:250,0],y_test[200:250,5])
plt.scatter(y_test_generated[200:250,0],y_test[200:250,5])
plt.xlim(1e-12,13e-12)
plt.show()

In [ ]:
pred_train = model(X_train_normalized.to(device))
output_train = np.array(pred_train.tolist())
mean_output_train = output_train*tr_y_std + tr_y_mean

In [ ]:
input_for_G = torch.cat((sample_z(len(noise_train), dim_input), torch.from_numpy(np.repeat(X_train_normalized.numpy(), 50, axis=0))),axis=1)
noise_train_normalized_predicted = G(input_for_G).detach().cpu().numpy()
noise_train_predicted = noise_train_normalized_predicted*tr_noise_std+tr_noise_mean

In [ ]:
Y_train_predicted = np.repeat(mean_output_train,50,axis=0)+noise_train_predicted

In [ ]:
plt.scatter(y_train[50:100,0],y_train[50:100,5])
plt.scatter(Y_train_predicted[50:100,0],Y_train_predicted[50:100,5])
plt.xlim(1e-12,13e-12)
plt.show()

In [ ]:
plt.scatter(y_train[150:200,0],y_train[150:200,5])
plt.xlim(1e-12,13e-12)
plt.xlabel('Ioff,sat', fontsize=20)
plt.ylabel('Vt,sat', fontsize=20)
plt.show()

In [ ]:
plt.scatter(y_train[150:200,2],y_train[150:200,6])
plt.xlim(1e-5,13e-12)
plt.xlabel('Id,lin', fontsize=20)
plt.ylabel('Vt,lin', fontsize=20)
plt.show()

In [ ]:
X_test

In [ ]:
X_train

In [ ]:
plt.scatter(y_train[150:200,0],y_train[150:200,5])
plt.scatter(Y_train_predicted[150:200,0],Y_train_predicted[150:200,5])
plt.xlim(1e-12,13e-12)
plt.show()

In [ ]:
plt.style.use(‘default’)
for i in range(30):
    plt.figure()
    plt.scatter(Y_test[i*10:i*10+10][:,0], Y_test[i*10:i*10+10][:,2], label = ‘True’)
    plt.scatter(Y_output[i*10:i*10+10][:,0], Y_output[i*10:i*10+10][:,2], label = ‘Generated(BNN)‘)
    plt.title(r’$\sigma: %.4f, \xi_x: %.4f, \xi_y: %.4f, \alpha:%.4f$’%(X_all[i*10][0],X_all[i*10][1],X_all[i*10][2],X_all[i*10][3]))
    plt.xlim(1e-12,13e-12)
    plt.xlabel(‘Ioff,sat’, fontsize=20)
    plt.ylabel(‘Vt,sat’, fontsize=20)
    plt.legend(loc=‘lower left’, fontsize=15)
    if not os.path.exists(‘./figures’):
        os.makedirs(‘./figures’)
    plt.savefig(‘figures/Linear_%d.pdf’%i, bbox_inches=‘tight’)

In [ ]:
# V_T_sat
# I_off

In [ ]:
# Conditional VAE
# Bayesian Linear Regressison -> multivariate x
# Target을 mean, cov (9 by 9)


In [ ]:
# CGAN, WGAN
# infoGAN (실험 - attribute 바꿔보면서 의도대로 .. 각 attribute 잘 학습하지 않을까 ?)
# interpolation ..  